<a href="https://colab.research.google.com/github/sgoodfriend/rl-algo-impls/blob/main/rl_algo_impls/microrts/colab_microrts_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Python MicroRTS submission in Google Colab
[sgoodfriend/rl-algo-impls](https://github.com/sgoodfriend/rl-algo-impls) was used to [train a Python agent to play MicroRTS](https://wandb.ai/sgoodfriend/rl-algo-impls-benchmarks/reports/rl-algo-impls-MicroRTS-Training--Vmlldzo0NjA2NTAy). [Farama-Foundation/MicroRTS-Py](https://github.com/Farama-Foundation/MicroRTS-Py) was the inspiration for this training.

Given MicroRTS is a Java environment, a Python server runs the Pytorch model, which communicates over a socket connection. This Colab notebook demonstrates what is necessary to install the necessary Python dependencies and expose a command-line endpoint for the submission jar file to communicate with.

## Steps to run RAIRoundRobinTournament

1. Download the `rai.jar` submission file and `rl_algo_impls-0.0.16-py3-none-any.whl` Python installation file:

In [ ]:
%%capture
!apt-get install -y subversion
!svn export https://github.com/sgoodfriend/rl-algo-impls/tags/v0.0.16/rl_algo_impls/microrts/java/rai.jar
!wget https://github.com/sgoodfriend/rl-algo-impls/releases/download/v0.0.16/rl_algo_impls-0.0.16-py3-none-any.whl

2. Install dependencies and rl_algo_impls Python wheel. This exposes the `rai_microrts` command-line endpoint, which will be called by the jar file. Ubuntu 20.04's default JDK is Java 11. setuptools and wheel have to be downgraded for the installation because rl-algo-impls depends on stable-baslines3, which depends on gym 0.21.

In [ ]:
%%capture
# Output hidden for cleanliness; however, remove %%capture if any downstream issues
!apt install -y default-jdk
!pip install setuptools==65.5.0
!pip install wheel==0.38.4
!python -m pip install rl_algo_impls-0.0.16-py3-none-any.whl

3. Download the jar files (microrts.jar and bot jars) and maps to run the tournament:

In [ ]:
%%capture
# microrts.jar and bot jars
!svn export https://github.com/sgoodfriend/rl-algo-impls/tags/v0.0.16/rl_algo_impls/microrts/java/lib
# Maps
!svn export https://github.com/sgoodfriend/rl-algo-impls/tags/v0.0.16/rl_algo_impls/microrts/java/maps

3. Run `tournaments.RAIRoundRobinTournament` which runs 10 iterations of the rai.jar `RAISocketAI` against 6 opponents on a 16x16 map. Depending on hardware, `RAISocketAI` can average from 50ms per turn on a Colab standard machine with 2-2.2GHz CPUs to 10ms per turn on modern CPUs, such as an Apple M1Max.

In [ ]:
!java -cp "rai.jar:$(find lib -name "*.jar" | tr '\n' ':')" tournaments.RAIRoundRobinTournament

## Performance Discussion
This Colab notebook demonstrates that a GPU isn't required to run this agent. *However, if a GPU isn't available, a relatively beefy CPU is instead required for a model of this size to reliably compete under the 100ms turn timelimit.* The model has 32 convolutional layers and about 5 million parameters. So, beefy, but still smaller than something like a resnet50.

PyTorch uses CPU multithreading to speedup the inference, so CPU clock speed and available cores are both helpful. For example, here's the performance on hardware I've tested so far:

| CPU          | Threads | Clock Speed | 16x16 avg turn | 64x64 avg turn |
| -------------| --------| ------------| ---------------| ---------------|
| Colab        | 2       | 2.2 GHz     | 30-60 ms       | 450 ms         |
| Apple M1 Max | 8+2     | 3.2/2.2 GHz | 16 ms          | 60-65 ms       |
| Xeon 8358    | 30*     | 2.6 GHz     | 11 ms          | 25 ms          |

*CPU multithreading is capped to 16 threads in PyTorch

Occassionally, the agent will exceed 100 ms, even if the average is significantly below 100 ms. This happens more if there's other activity happening on the machine (for example, me using my own computer).

If the long computations are rare (less than once per game), a timeout on the Java side to submit an empty turn if the Python server doesn't respond within 100 ms is a good safety valve, though it could mean that some CPU cores are taken up by the Python server on the opponent's turn (killing the Python process instead would lead to considerable delay starting the server and loading the PyTorch model).

***However, if the competition machines are closer to the Colab machines, then a significantly smaller model is required.***